In [21]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten, Conv2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sklearn
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import zipfile
from tensorflow.keras.preprocessing import image
import os

In [22]:
trainDIR = '/workspace/seg_train/seg_train/'
testDIR = '/workspace/seg_test/seg_test/'
predDIR = '/workspace/seg_pred/seg_pred'

In [23]:
def get_images(directory):
    Images=[]
    Labels=[]
    
    for labels in os.listdir(directory):
        if labels == 'glacier':
            label = 0
        elif labels == 'sea':
            label = 1
        elif labels == 'buildings':
            label = 2
        elif labels == 'forest':
            label = 3
        elif labels == 'mountain':
            label = 4
        elif labels == 'street':
            label = 5
        
        for file in os.listdir(directory+labels):
            im =  image.load_img(directory+labels+r'/'+file, target_size=(150,150))
            img = image.img_to_array(im)
            Images.append(img)
            Labels.append(label)
    
    Images, Labels = sklearn.utils.shuffle(Images, Labels, random_state=14)
    return Images, Labels

In [24]:
Images, Labels = get_images(trainDIR)
Images = np.array(Images)
Labels = np.array(Labels)

In [25]:
def create_model(lr=0.01, opt = keras.optimizers.Adam(),
                 nn1 = 32, nn2 = 100):
                                                     
    model = Sequential()
    
    model.add(Convolution2D(nn1,(3,3), input_shape=(150,150,3), activation='relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Flatten())
    model.add(Dense(nn2, activation='relu'))
    model.add(Dropout(0))   
    model.add(Dense(6, activation='softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model


In [26]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# model class to use in the scikit random search CV 
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=5)

In [27]:
# learning rate parameters
lr=[1e-2, 1e-3]

# neurons in each layer
nn1 = [32,64]
nn2 = [100,200]

# optimizer
optimizer = ['adam','rmsprop']

# dictionary summary
param_grid = dict(nn1= nn1, nn2 = nn2, lr=lr, opt=optimizer)

In [28]:
from sklearn.model_selection import RandomizedSearchCV, KFold

grid = RandomizedSearchCV(estimator=model, cv=KFold(3), param_distributions=param_grid, n_iter=3, n_jobs=1)

In [29]:
grid_result = grid.fit(Images, Labels)

Epoch 1/10
1872/1872 [==============================] - 12s 6ms/step - loss: 38.0760 - accuracy: 0.4330
Epoch 2/10
1872/1872 [==============================] - 12s 6ms/step - loss: 0.6517 - accuracy: 0.7915
Epoch 3/10
1872/1872 [==============================] - 12s 6ms/step - loss: 0.4038 - accuracy: 0.8888
Epoch 4/10
1872/1872 [==============================] - 12s 7ms/step - loss: 0.5727 - accuracy: 0.8580
Epoch 5/10
1872/1872 [==============================] - 12s 7ms/step - loss: 0.5842 - accuracy: 0.8625
Epoch 6/10
1872/1872 [==============================] - 12s 6ms/step - loss: 0.3200 - accuracy: 0.9197
Epoch 7/10
1872/1872 [==============================] - 12s 7ms/step - loss: 0.3041 - accuracy: 0.9290
Epoch 8/10
1872/1872 [==============================] - 12s 7ms/step - loss: 0.2890 - accuracy: 0.9302
Epoch 9/10
1872/1872 [==============================] - 12s 7ms/step - loss: 0.2945 - accuracy: 0.9434
Epoch 10/10
936/936 [==============================] - 3s 4ms/step - los

In [30]:
print(grid_result.best_params_)

{'opt': 'rmsprop', 'nn2': 200, 'nn1': 64, 'lr': 0.001}


In [31]:
best_model = grid_result.best_estimator_.model

In [32]:
test, test_labels = get_images(testDIR)
test = np.array(test)
test_labels = np.array(test_labels)

In [33]:
best_model.evaluate(test,test_labels)

94/94 [==============================] - 1s 7ms/step - loss: 34.8607 - accuracy: 0.5457


[34.8607063293457, 0.5456666946411133]